In [1]:
### create pickle file with SAX dimer features
## PAA time window = 0.05 #0.2s  # 0.5, 0.1s

# change break-points & dimers for 3-letter alphabet

import pandas as pd
import numpy as np
import os
import pickle
#import re
import regex as re
import matplotlib.pyplot as plt

import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [2]:
## signal cleaning: removing baseline (set mean = 0) & line frequency
from scipy import signal 
#notch_filter(clean(x['wake'][k]),50)  ## line freq = 50Hz

def clean(eeg):
    mean = np.mean(eeg)
    eeg = [x-mean for x in eeg]
    return eeg

def notch_filter(eeg,f0):
    fs = 100 #256  ## sampling frequency
    Q = 30.0  # Quality factor
    w0 = f0/(fs/2)  # Normalized Frequency
    [b1, a1] = signal.iirnotch(w0, Q)
    notched_signal = signal.filtfilt(b1, a1, eeg) # Apply filter
    return notched_signal


In [3]:
## create SAX string representation & dimer features

# STEPS
# for each epoch:
# 1. normalize the data over a window (epoch size)
# 2. convert to PAA - take small segments & represent by the average over the segment
# 3. choose alphabet size & the levels/ threshold for discretisation
# 4. convert to string representation
# 5. vector representation - numbers of different dimers

# discretise over 0.25S = 0.25 * 100 = 25 points; reduces to 120 points
#               0.5s => 50 points; reduces to 60 points
#               0.1 s = 0.1 * 100 = 10 points -> 300 data points
#               0.05s = 5 points -> 600 data points


### 1. normalise the data, 1 epoch (30s, i.e. 3000 data points) at a time ###
# if the SD is very small (less than a threshold), do not normalize the data
def z_norm(y):
    sd = y.std()
    if sd < 0.01:
        y = y
    else:
        y =(y - y.mean())/sd
    
    return y



#### 2. convert to PAA - take a window & represent by the average over the window ####
# discretise over 0.25S = 0.25 * 100 = 25 points  -> results in 120 data points
#               0.5s = 50 points   -> results in 60 data points
#               0.1 s = 0.1 * 100 = 10 points -> 300 data points
#               0.05s = 5 points -> 600 data points
#time_window = 0.5
def get_paa(y, time_window):
    # convert time_window from sec to no. of data points
    time_window = time_window * 100  # 100 is sampling freq
    
    paa = []
    for i in range(0, int(len(y)/time_window)): # len(y)/time_window gives the no. of steps
        # take a segment of data
        arr = y[(i * int(time_window)):((i+1) * int(time_window))]
        # convert to mean & append to list
        paa.append(arr.mean())
    return paa



#### 3. choose alphabet size & the levels/ threshold for discretisation ####
### set this up as a dictionary to choose different alphabet sizes  ###
# let alphabet size = 4;
# cuts: [-0.67, 0, 0.67]

# for alphabet size = 3; cuts: [-0.43, 0.43]
# alphabet size 5; cuts : [-0.84, -0.25, 0.25, 0.84]


#### 4. convert to string  ####
def get_sax_string(paa):
    sax = []
    
    for i in range(0,len(paa)):
        '''
        ## 4 alphabet ##
        if paa[i] <= -0.67:
            alpha = 'a'
        elif (paa[i] > -0.67 and paa[i] <=0):
            alpha = 'b'
        elif (paa[i] > 0 and paa[i] <=0.67):
            alpha = 'c'
        else:
            alpha = 'd'
        #'''
        '''
        ## 3 alphabet ##
        if paa[i] <= -0.43:
            alpha = 'a'
        elif (paa[i] > -0.43 and paa[i] <=0.43):
            alpha = 'b'
        else:
            alpha = 'c'
        #'''    
        #'''
        ## 5 alphabet ##
        if paa[i] <= -0.84:
            alpha = 'a'
        elif (paa[i] > -0.84 and paa[i] <= -0.25):
            alpha = 'b'
        elif (paa[i] > -0.25 and paa[i] <= 0.25):
            alpha = 'c'
        elif (paa[i] > 0.25 and paa[i] <= 0.84):
            alpha = 'd'
        else:
            alpha = 'e'
        #'''
        
        sax.append(alpha)
    
    # convert sax from list to string format
    sax_string = ''.join(sax)
    return sax_string



#### 5. vector representation - numbers of different dimers ####
# different types of dimers
# aa ab ac ad
# ba bb bc bd
# ca cb cc cd
# da db dc dd
### ways to count ###
# sax_string.count(dim_alpha[i]) => 'bbb' : counts bb as 1  => FIX THIS
# len(re.findall(dim_alpha[i], sax_string)) : counts bb as 1

'''
###  set up a list of the dimers possible  ###
#dim_alpha = ['aa','ab','ac','ad','ba','bb','bc','bd','ca','cb','cc','cd','da','db','dc','dd']
#alphabets = ['a', 'b', 'c']
alphabets = ['a', 'b', 'c', 'd', 'e']
dim_alpha = []
for i in range(len(alphabets)):
    for j in range(len(alphabets)):
        dim_alpha.append(alphabets[i] + alphabets[j])
print(dim_alpha)
#'''
## setting up trimer list
#alphabets = ['a', 'b', 'c']
#alphabets = ['a', 'b', 'c', 'd']
alphabets = ['a', 'b', 'c', 'd', 'e']
dim_alpha = []
for i in range(len(alphabets)):
    for j in range(len(alphabets)):
        for k in range(len(alphabets)):
            dim_alpha.append(alphabets[i] + alphabets[j] + alphabets[k])
print(dim_alpha)
#'''


def get_sax_dimer_array(sax_string):
    dim_arr = []
    
    for i in range(0, len(dim_alpha)):
        dim_arr.append(len(re.findall(dim_alpha[i], sax_string, overlapped=True)))
    #https://stackoverflow.com/questions/2970520/string-count-with-overlapping-occurrences
    
    #print(dim_arr)
    #ncols = 4
    #im = np.reshape(dim_arr,(-1,ncols)) #x.reshape(1754, 1754)
    #plt.gray()
    #plt.imshow(im)
    return dim_arr



def run_all(y, time_window):
    y = z_norm(y)
    #time_window = 0.5
    paa = get_paa(y, time_window)
    sax_string = get_sax_string(paa)
    sax_feature = get_sax_dimer_array(sax_string)
    ## added the following 2 lines to make the data fraction
    total = sum(sax_feature)
    sax_feature = [x/total for x in sax_feature]
    return sax_feature

['aaa', 'aab', 'aac', 'aad', 'aae', 'aba', 'abb', 'abc', 'abd', 'abe', 'aca', 'acb', 'acc', 'acd', 'ace', 'ada', 'adb', 'adc', 'add', 'ade', 'aea', 'aeb', 'aec', 'aed', 'aee', 'baa', 'bab', 'bac', 'bad', 'bae', 'bba', 'bbb', 'bbc', 'bbd', 'bbe', 'bca', 'bcb', 'bcc', 'bcd', 'bce', 'bda', 'bdb', 'bdc', 'bdd', 'bde', 'bea', 'beb', 'bec', 'bed', 'bee', 'caa', 'cab', 'cac', 'cad', 'cae', 'cba', 'cbb', 'cbc', 'cbd', 'cbe', 'cca', 'ccb', 'ccc', 'ccd', 'cce', 'cda', 'cdb', 'cdc', 'cdd', 'cde', 'cea', 'ceb', 'cec', 'ced', 'cee', 'daa', 'dab', 'dac', 'dad', 'dae', 'dba', 'dbb', 'dbc', 'dbd', 'dbe', 'dca', 'dcb', 'dcc', 'dcd', 'dce', 'dda', 'ddb', 'ddc', 'ddd', 'dde', 'dea', 'deb', 'dec', 'ded', 'dee', 'eaa', 'eab', 'eac', 'ead', 'eae', 'eba', 'ebb', 'ebc', 'ebd', 'ebe', 'eca', 'ecb', 'ecc', 'ecd', 'ece', 'eda', 'edb', 'edc', 'edd', 'ede', 'eea', 'eeb', 'eec', 'eed', 'eee']


In [4]:
#time_window = 0.1 #0.5 #0.25

def get_PSD_from_pickle(pid,day,x):
    df = []
    eeg_class = []
    epochnum = []

    
    for k in list(x['wake'].keys()):
        label = 'wake'  #seg = k.split('e')[0]
        epoch = k
        y = x['wake'][k]   ## 
        #y = notch_filter(clean(x['wake'][k]),50) ##
        df.append(run_all(y, time_window))  #x['wake'][k])
        eeg_class.append(label)
        epochnum.append(k)

    for k in list(x['sleep_stage_1'].keys()):
        label = 'sleep_stage_1'   #seg = k.split('e')[0]
        epoch = k
        y = x['sleep_stage_1'][k]   ## 
        #y = notch_filter(clean(x['sleep_stage_1'][k]),50) ##
        df.append(run_all(y, time_window))  #x['sleep_stage_1'][k])
        eeg_class.append(label)
        epochnum.append(k)
            
    for k in list(x['sleep_stage_2'].keys()):
        label = 'sleep_stage_2'   #seg = k.split('e')[0]
        epoch = k
        y = x['sleep_stage_2'][k]   ## 
        #y = notch_filter(clean(x['sleep_stage_2'][k]),50) ##
        df.append(run_all(y, time_window))  #x['sleep_stage_2'][k])
        eeg_class.append(label)
        epochnum.append(k)

    #print(df)    
    df = pd.DataFrame(df)  ## also, add subject ID
    print(df)
    df['pID'] = pid
    df['day'] = day
    eeg_class = pd.DataFrame(eeg_class, columns=['class_label'])
    epochnum = pd.DataFrame(epochnum, columns=['epoch'])
    
    df = pd.concat([df,epochnum,eeg_class],axis=1)
    
    return df

In [5]:
file_location='C:/Users/DAR9KOR/Desktop/data/HEALTHCARE/EEG/data/eeg_epochs_dec12_2018/raw_data' #C:\\Users\\DAR9KOR\\Desktop\\data\\HEALTHCARE\\EEG\\data\\eeg_epochs_dec12_2018'
os.chdir('C:/Users/DAR9KOR/Desktop/data/HEALTHCARE/EEG/data/eeg_epochs_dec12_2018/raw_data') #'/home/intern_eyecare/Desktop/EEG/raw_data') #C:\\Users\\DAR9KOR\\Desktop\\data\\HEALTHCARE\\EEG\\data\\eeg_epochs_dec12_2018')

files=os.listdir(file_location)
file_list = [f for f in files if f.endswith('.pickle')]   # .txt  # .csv

In [10]:
time_window = 0.2   #0.05 #0.1 # 0.2  #0.5 #0.25 
dfull = pd.DataFrame()

for file_val in file_list:
    #print(file_val)
    patient = file_val[2:5]
    day = file_val[5:6]
    
    with open(file_val, 'rb') as f:
        print(file_val)
        x = pickle.load(f)      
        dat = get_PSD_from_pickle(patient,day,x)
    f.close()
        
    dfull = pd.concat([dfull, dat],ignore_index=True)

SC4001E0-PSG-EEG.pickle
           0         1         2         3         4         5         6    \
0     0.054054  0.000000  0.013514  0.006757  0.000000  0.006757  0.006757   
1     0.013514  0.020270  0.013514  0.000000  0.006757  0.006757  0.006757   
2     0.060811  0.013514  0.013514  0.006757  0.000000  0.000000  0.006757   
3     0.020270  0.006757  0.006757  0.000000  0.000000  0.013514  0.000000   
4     0.000000  0.013514  0.006757  0.000000  0.000000  0.000000  0.000000   
5     0.013514  0.027027  0.000000  0.006757  0.000000  0.000000  0.020270   
6     0.027027  0.013514  0.013514  0.006757  0.000000  0.006757  0.013514   
7     0.054054  0.027027  0.006757  0.000000  0.000000  0.006757  0.006757   
8     0.000000  0.006757  0.006757  0.000000  0.000000  0.000000  0.000000   
9     0.040541  0.020270  0.006757  0.000000  0.000000  0.000000  0.006757   
10    0.020270  0.020270  0.027027  0.000000  0.000000  0.000000  0.006757   
11    0.047297  0.027027  0.013514  0.00

           0         1         2         3         4         5         6    \
0     0.033784  0.020270  0.006757  0.013514  0.006757  0.000000  0.006757   
1     0.054054  0.027027  0.006757  0.000000  0.000000  0.006757  0.013514   
2     0.033784  0.027027  0.013514  0.000000  0.000000  0.013514  0.033784   
3     0.040541  0.013514  0.000000  0.000000  0.006757  0.000000  0.013514   
4     0.006757  0.033784  0.006757  0.000000  0.000000  0.006757  0.020270   
5     0.020270  0.020270  0.000000  0.006757  0.000000  0.000000  0.020270   
6     0.047297  0.027027  0.013514  0.000000  0.000000  0.006757  0.013514   
7     0.027027  0.033784  0.006757  0.000000  0.000000  0.000000  0.013514   
8     0.027027  0.013514  0.020270  0.000000  0.000000  0.000000  0.013514   
9     0.006757  0.040541  0.000000  0.000000  0.000000  0.006757  0.013514   
10    0.033784  0.006757  0.013514  0.000000  0.000000  0.000000  0.006757   
11    0.054054  0.020270  0.020270  0.000000  0.000000  0.000000

           0         1         2         3         4         5         6    \
0     0.013514  0.013514  0.000000  0.000000  0.000000  0.006757  0.006757   
1     0.033784  0.006757  0.000000  0.006757  0.013514  0.000000  0.006757   
2     0.054054  0.020270  0.013514  0.000000  0.000000  0.000000  0.013514   
3     0.047297  0.000000  0.013514  0.006757  0.000000  0.006757  0.013514   
4     0.020270  0.013514  0.020270  0.006757  0.000000  0.000000  0.027027   
5     0.013514  0.027027  0.000000  0.000000  0.000000  0.000000  0.013514   
6     0.013514  0.020270  0.006757  0.000000  0.013514  0.006757  0.020270   
7     0.040541  0.013514  0.000000  0.000000  0.006757  0.000000  0.000000   
8     0.027027  0.013514  0.006757  0.000000  0.000000  0.000000  0.006757   
9     0.027027  0.013514  0.000000  0.000000  0.000000  0.027027  0.006757   
10    0.047297  0.006757  0.006757  0.000000  0.013514  0.000000  0.000000   
11    0.020270  0.020270  0.006757  0.000000  0.020270  0.013514

           0         1         2         3         4         5         6    \
0     0.006757  0.027027  0.013514  0.000000  0.000000  0.000000  0.013514   
1     0.000000  0.013514  0.006757  0.000000  0.006757  0.006757  0.020270   
2     0.027027  0.027027  0.006757  0.006757  0.000000  0.013514  0.000000   
3     0.040541  0.013514  0.006757  0.000000  0.006757  0.006757  0.020270   
4     0.027027  0.033784  0.000000  0.000000  0.006757  0.000000  0.020270   
5     0.020270  0.027027  0.006757  0.000000  0.000000  0.013514  0.013514   
6     0.027027  0.033784  0.013514  0.000000  0.000000  0.013514  0.013514   
7     0.054054  0.040541  0.006757  0.000000  0.000000  0.027027  0.013514   
8     0.006757  0.033784  0.006757  0.000000  0.000000  0.006757  0.033784   
9     0.006757  0.013514  0.013514  0.000000  0.000000  0.000000  0.013514   
10    0.027027  0.027027  0.006757  0.013514  0.000000  0.006757  0.020270   
11    0.020270  0.006757  0.013514  0.006757  0.000000  0.006757

           0         1         2         3         4         5         6    \
0     0.020270  0.006757  0.000000  0.006757  0.013514  0.000000  0.000000   
1     0.047297  0.027027  0.000000  0.006757  0.000000  0.000000  0.020270   
2     0.000000  0.020270  0.006757  0.000000  0.000000  0.020270  0.020270   
3     0.027027  0.020270  0.000000  0.000000  0.000000  0.000000  0.020270   
4     0.040541  0.027027  0.000000  0.000000  0.006757  0.027027  0.000000   
5     0.020270  0.027027  0.006757  0.000000  0.000000  0.000000  0.013514   
6     0.027027  0.013514  0.000000  0.000000  0.006757  0.006757  0.013514   
7     0.033784  0.033784  0.000000  0.000000  0.000000  0.020270  0.020270   
8     0.033784  0.013514  0.000000  0.000000  0.000000  0.013514  0.033784   
9     0.040541  0.006757  0.000000  0.013514  0.000000  0.000000  0.000000   
10    0.013514  0.020270  0.000000  0.000000  0.006757  0.006757  0.020270   
11    0.047297  0.033784  0.000000  0.000000  0.000000  0.006757

...

           0         1         2         3         4         5         6    \
0     0.013514  0.006757  0.006757  0.000000  0.013514  0.027027  0.006757   
1     0.013514  0.020270  0.000000  0.000000  0.000000  0.006757  0.033784   
2     0.006757  0.006757  0.000000  0.000000  0.000000  0.000000  0.000000   
3     0.006757  0.006757  0.006757  0.000000  0.000000  0.006757  0.006757   
4     0.013514  0.013514  0.006757  0.006757  0.000000  0.006757  0.006757   
5     0.013514  0.013514  0.013514  0.000000  0.000000  0.000000  0.006757   
6     0.000000  0.020270  0.006757  0.000000  0.000000  0.006757  0.006757   
7     0.000000  0.013514  0.013514  0.000000  0.000000  0.000000  0.006757   
8     0.013514  0.006757  0.006757  0.000000  0.000000  0.006757  0.000000   
9     0.013514  0.013514  0.000000  0.000000  0.000000  0.000000  0.020270   
10    0.000000  0.013514  0.000000  0.000000  0.000000  0.000000  0.013514   
11    0.000000  0.013514  0.006757  0.000000  0.000000  0.000000

           0         1         2         3         4         5         6    \
0     0.033784  0.020270  0.006757  0.000000  0.006757  0.027027  0.006757   
1     0.033784  0.020270  0.013514  0.006757  0.006757  0.006757  0.000000   
2     0.000000  0.013514  0.006757  0.006757  0.000000  0.000000  0.013514   
3     0.006757  0.000000  0.006757  0.000000  0.000000  0.006757  0.000000   
4     0.020270  0.013514  0.000000  0.000000  0.006757  0.000000  0.000000   
5     0.020270  0.006757  0.006757  0.000000  0.000000  0.000000  0.020270   
6     0.013514  0.013514  0.000000  0.013514  0.000000  0.000000  0.020270   
7     0.013514  0.013514  0.000000  0.000000  0.006757  0.000000  0.013514   
8     0.047297  0.027027  0.006757  0.006757  0.000000  0.006757  0.000000   
9     0.047297  0.013514  0.006757  0.006757  0.000000  0.013514  0.006757   
10    0.020270  0.027027  0.000000  0.006757  0.000000  0.006757  0.006757   
11    0.047297  0.020270  0.013514  0.000000  0.000000  0.000000

           0         1         2         3         4         5         6    \
0     0.033784  0.020270  0.006757  0.000000  0.000000  0.006757  0.020270   
1     0.033784  0.033784  0.006757  0.000000  0.000000  0.020270  0.013514   
2     0.006757  0.020270  0.006757  0.000000  0.000000  0.020270  0.013514   
3     0.006757  0.006757  0.006757  0.000000  0.006757  0.000000  0.013514   
4     0.000000  0.000000  0.006757  0.000000  0.006757  0.013514  0.000000   
5     0.013514  0.006757  0.000000  0.000000  0.000000  0.000000  0.020270   
6     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7     0.006757  0.006757  0.000000  0.000000  0.000000  0.006757  0.000000   
8     0.000000  0.006757  0.000000  0.000000  0.000000  0.006757  0.006757   
9     0.013514  0.006757  0.000000  0.006757  0.006757  0.000000  0.013514   
10    0.006757  0.006757  0.000000  0.000000  0.000000  0.006757  0.020270   
11    0.000000  0.006757  0.006757  0.000000  0.006757  0.006757

           0         1         2         3         4         5         6    \
0     0.006757  0.013514  0.020270  0.006757  0.000000  0.000000  0.006757   
1     0.020270  0.020270  0.000000  0.013514  0.000000  0.006757  0.013514   
2     0.054054  0.027027  0.013514  0.000000  0.000000  0.000000  0.006757   
3     0.047297  0.013514  0.013514  0.000000  0.000000  0.013514  0.000000   
4     0.020270  0.027027  0.006757  0.000000  0.000000  0.020270  0.013514   
5     0.020270  0.006757  0.013514  0.013514  0.006757  0.000000  0.006757   
6     0.013514  0.020270  0.013514  0.006757  0.000000  0.000000  0.000000   
7     0.020270  0.006757  0.013514  0.000000  0.000000  0.000000  0.006757   
8     0.033784  0.033784  0.006757  0.000000  0.006757  0.000000  0.027027   
9     0.054054  0.013514  0.006757  0.000000  0.006757  0.013514  0.013514   
10    0.020270  0.027027  0.006757  0.006757  0.006757  0.000000  0.006757   
11    0.006757  0.006757  0.006757  0.006757  0.000000  0.013514

           0         1         2         3         4         5         6    \
0     0.006757  0.013514  0.000000  0.000000  0.000000  0.013514  0.013514   
1     0.060811  0.020270  0.006757  0.013514  0.000000  0.000000  0.006757   
2     0.020270  0.013514  0.006757  0.000000  0.006757  0.000000  0.006757   
3     0.020270  0.013514  0.013514  0.006757  0.000000  0.000000  0.013514   
4     0.006757  0.020270  0.006757  0.000000  0.000000  0.000000  0.006757   
5     0.006757  0.013514  0.020270  0.000000  0.000000  0.000000  0.006757   
6     0.040541  0.033784  0.006757  0.000000  0.000000  0.013514  0.006757   
7     0.020270  0.013514  0.000000  0.006757  0.006757  0.006757  0.020270   
8     0.033784  0.006757  0.013514  0.006757  0.000000  0.000000  0.000000   
9     0.020270  0.020270  0.000000  0.006757  0.000000  0.000000  0.027027   
10    0.000000  0.013514  0.000000  0.000000  0.000000  0.000000  0.020270   
11    0.013514  0.013514  0.000000  0.000000  0.000000  0.000000

In [11]:
## save data
pickle.dump(dfull, open('Frac_sax_trimer_0.2s_5alpha_EEG_all_patients.pkl','wb')) # 0.2, 0.1

#pickle.dump(dfull, open('Frac_sax_dimer_0.1s_filter_4alpha_EEG_all_patients.pkl','wb')) # 0.2, 0.1
#pickle.dump(dfull, open('Frac_sax_dimer_0.2s_EEG_all_patients.pkl','wb'))